In [2]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [6]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [7]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

In [8]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/10
363/363 [==============================] - 3s 4ms/step - loss: 2.3520 - output_1_loss: 2.1808 - output_2_loss: 3.8930 - val_loss: 1.0313 - val_output_1_loss: 0.8600 - val_output_2_loss: 2.5733
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.9465 - output_1_loss: 0.8069 - output_2_loss: 2.2031 - val_loss: 0.8248 - val_output_1_loss: 0.7148 - val_output_2_loss: 1.8144
Epoch 3/10
363/363 [==============================] - 1s 2ms/step - loss: 0.8220 - output_1_loss: 0.7232 - output_2_loss: 1.7104 - val_loss: 0.7568 - val_output_1_loss: 0.6705 - val_output_2_loss: 1.5337
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.7663 - output_1_loss: 0.6848 - output_2_loss: 1.5002 - val_loss: 0.7183 - val_output_1_loss: 0.6428 - val_output_2_loss: 1.3977
Epoch 5/10
363/363 [==============================] - 1s 2ms/step - loss: 0.7309 - output_1_loss: 0.6577 - output_2_loss: 1.3896 - val_loss: 0.6890 - val_output_1_loss: 0.6189 - val_ou